In [2]:
%load_ext autoreload
%autoreload 2
import requests

import requests
import json

# base url
base_url = "http://api:5000/istsos4/v1.1/"

# Headers (optional, but specifying Content-Type ensures proper handling of JSON data)
headers = {
    'Content-Type': 'application/json'
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Creation Order
The SensorThingsAPI data schema has relations.  
So to respect references among elements you have to follow a defined order in creating them.  
The usual order is:

1. Thing
2. Location
3. ObservedProperty
4. Sensor
5. Datastream
6. Observation (+FeatureOfInterest)


### Thing creation

In [3]:
body = {
  "name" : "MAG_LOD6",
  "description" : "Maggia river in Lodrino",
  "properties" : {
    "subcatchment" : "Maggia",
    "subcatchment_area" : 4000,
    "subcatchment_number": 12345
  }
}

# POST request with the JSON body
response = requests.post(base_url + 'Things', data=json.dumps(body), headers=headers)

In [4]:
# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print("Thing created successfully.")
    print(response.headers['location'])
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Thing created successfully.
http://api:5000/istsos4/v1.1/Things(33)


In [5]:
# Let's get the Things @iot.id usimng a regex to extract the number in parentheses
import re
match = re.search(r'\((\d+)\)', response.headers['location'])

if match:
    things_id = int(match.group(1))
    print(f"Number extracted: {things_id}")
else:
    print("No number found in parentheses.")

Number extracted: 33


### Create an associated Location

In [8]:
body = {
  "name": "Location of the MAG_LOD2 gauge",
  "description": "The gauge is located close to the bridge",
  "properties": {},
  "encodingType": "application/geo+json",
  "location": {
    "type": "Point",
    "coordinates": [8.10, 50.00]
  },
  "Things": [
    { "@iot.id": things_id}
  ]
}

print(json.dumps(body))

# POST request with the JSON body
response = requests.post(base_url + 'Locations', data=json.dumps(body), headers=headers)

{"name": "Location of the MAG_LOD2 gauge", "description": "The gauge is located close to the bridge", "properties": {}, "encodingType": "application/geo+json", "location": {"type": "Point", "coordinates": [8.1, 50.0]}, "Things": [{"@iot.id": 33}]}


In [9]:
# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print("Location created successfully.")
    print(response.headers['location'])
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Location created successfully.
http://api:5000/istsos4/v1.1/Locations(14)


In [10]:
print(f"{base_url}Things({things_id})?$expand=Locations")

http://api:5000/istsos4/v1.1/Things(33)?$expand=Locations


In [11]:
from pprint import pprint
response = requests.get(f"{base_url}Things({things_id})?$expand=Locations")
pprint(response.json())

{'@iot.id': 33,
 '@iot.selfLink': 'http://api:5000/istsos4/v1.1/Things(33)',
 'Datastreams@iot.navigationLink': 'http://api:5000/istsos4/v1.1/Things(33)/Datastreams',
 'HistoricalLocations@iot.navigationLink': 'http://api:5000/istsos4/v1.1/Things(33)/HistoricalLocations',
 'Locations': [{'@iot.id': 14,
                '@iot.selfLink': 'http://api:5000/istsos4/v1.1/Locations(14)',
                'description': 'The gauge is located close to the bridge',
                'encodingType': 'application/geo+json',
                'location': {'coordinates': [8.1, 50], 'type': 'Point'},
                'name': 'Location of the MAG_LOD2 gauge',
                'properties': {}}],
 'Locations@iot.navigationLink': 'http://api:5000/istsos4/v1.1/Things(33)/Locations',
 'description': 'Maggia river in Lodrino',
 'name': 'MAG_LOD6',
 'properties': {'subcatchment': 'Maggia',
                'subcatchment_area': 4000,
                'subcatchment_number': 12345}}
